In [1]:
import json
import pprint
import pandas as pd
import numpy as np
import os
import subprocess
import pexpect
import sys
import time
import tldextract

In [2]:
usuarioBlackbird = 
senhaBlackbird = 
filesBlackbird = ['/scratch4/leandro/data/fn/pickles/top-10-perserver-20210817.csv',
                   '/scratch4/leandro/data/fn/compiled_trusty_news_portals.txt',
                   '/scratch4/leandro/data/fn/url2title.csv']
destinationLocal = './data/'

In [3]:
#Baixa os arquivos listados em filesBlackbird APENAS SE já não estiverem no diretorio /data
for file in filesBlackbird:
    if not os.path.isfile('data/' + file.split('/')[-1]):
        child = pexpect.spawn("bash")
        child.timeout= 300
        child.sendline("scp -P2512 -r " +
                              usuarioBlackbird+
                              "@blackbird.dcc.ufmg.br:"+
                              file+ 
                              " "+
                             destinationLocal)
        child.logfile_read = sys.stdout
        child.expect("password:")
        child.sendline(senhaBlackbird)
        child.expect("100%")
        
        time.sleep(2)
        child.close() 
        time.sleep(2)

scp -P2512 -r joaomcouto@blackbird.dcc.ufmg.br:/scratch4/leandro/data/fn/pickles/top-10-perserver-20210817.csv ./data/
(base) ]0;joaomcouto@SpectreUbuntu: ~/git/E02_featuresjoaomcouto@SpectreUbuntu:~/git/E02_features$ scp -P2512 -r joaomcouto@blackbird.dcc.ufmg.br:/scratch4/leandro/data/fn/pickles/top-10-perserver-20210817.csv ./data/
joaomcouto@blackbird.dcc.ufmg.br's password: 
top-10-perserver-20210817.csv                 100%  245KB   1.6MB/s   00:00    
scp -P2512 -r joaomcouto@blackbird.dcc.ufmg.br:/scratch4/leandro/data/fn/compiled_trusty_news_portals.txt ./data/
(base) ]0;joaomcouto@SpectreUbuntu: ~/git/E02_featuresjoaomcouto@SpectreUbuntu:~/git/E02_features$ scp -P2512 -r joaomcouto@blackbird.dcc.ufmg.br:/scratch4/leandro/data/fn/compiled_trusty_news_portals.txt ./data/
joaomcouto@blackbird.dcc.ufmg.br's password: 
compiled_trusty_news_portals.txt              100% 1884    66.2KB/s   00:00    
scp -P2512 -r joaomcouto@blackbird.dcc.ufmg.br:/scratch4/leandro/data/fn/url2tit

In [4]:
#Carrega as URLs das 10 noticias mais compartilhadas dos sites de baixa credibilidade em um dataframe
dfUrlFalse = pd.read_csv('data/top-10-perserver-20210817.csv', delimiter='|')
dfUrlFalse.head(1000)

,server,url,amount
0,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/notici...,517
1,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/notici...,461
2,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/notici...,452
3,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/notici...,420
4,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/notici...,411
...,...,...,...
995,canalgama.com.br,https://canalgama.com.br/milhares-vao-as-ruas-...,35
996,canalgama.com.br,https://canalgama.com.br/bolsonaro-denuncia-tr...,31
997,canalgama.com.br,https://canalgama.com.br/general-heleno-diz-qu...,25
998,canalgama.com.br,https://canalgama.com.br/joice-perde-quase-1-m...,19


In [5]:
dfUrlFalse.shape

(2052, 3)

In [6]:
#Carrega e pre-processa os dominios de alta credibilidade COMO LISTADO na anj https://www.anj.org.br/associados/)
#O pre-processamento consiste em:
    #(1) Pra cada listagem, transforma em subdominio.dominio.sufixo
          #Ou seja, 
          #bbc.com/portuguese vira bbc.com
          #theintercept.com/brasil/ vira theintercept.com
          #uol.com.br/ vira uol.com.br
    #(2) Para listagens que tem subdominio e.g. g1.globo.com (g1 é subdominio) adiciona o dominio à lista de confiaveis
          #Ou seja, 
          #além de valor.globo.com teremos tambem globo.com na lista de fonte confiaveis
    #(3) Ordena a lista e retira duplicatas
          #A lista original da ANJ já vem com redundancias e.g. uol.com.br e uol.com.br/

with open('data/compiled_trusty_news_portals.txt', 'r') as file :
    filedata = file.readlines()

extraMainTrustyDomains = []
for i,dom in enumerate(filedata):
    ext = tldextract.extract(dom)
    
    if(len(ext.subdomain)>0):
        filedata[i] = ext.subdomain + "." + ext.domain +'.'+ ext.suffix + "\n"
        extraMainTrustyDomains.append(ext.domain +'.'+ ext.suffix + "\n")
    else:
        filedata[i] = ext.domain +'.'+ ext.suffix + "\n"
            
filedata = filedata + extraMainTrustyDomains
filedata = sorted(list(set(filedata)))
    

with open('data/compiled_trusty_news_portals.txt', 'w') as file:
    file.write("".join(filedata))

trustyNewsDomains_ = open("data/compiled_trusty_news_portals.txt", "r")
trustyNewsDomains = pd.DataFrame([x.strip() for x in trustyNewsDomains_.readlines()], columns=['url'])
trustyNewsDomains.head(1000)

,url
0,acidadeon.com
1,acritica.com
2,agazetaderondoniadigital.com
3,agorarn.com.br
4,an.com.br
...,...
99,tribunapr.com.br
100,uai.com.br
101,uol.com.br
102,valor.globo.com


In [7]:
trustyNewsDomains.shape

(104, 1)

In [8]:
#Carrega a coluna URL do csv com todas as URLs expandidas pelo leandro e 
    #pre-processa para pegar apenas as URLs de sites de alta credibilidade
    
#O pre-processamento consiste em
     #(1) Consertar erros no csv: substitui aspas duplas por simples 
     #(2) Pega as URLs cujo dominio ou subdominio estão na lista da ANJ AMPLIADA conforme descrito acima

with open('data/url2title.csv', 'r') as file :
    filedata = file.read()
filedata = filedata.replace('"', "'")
with open('data/url2title.csv', 'w') as file:
    file.write(filedata)

dfUrlTrue_ = pd.read_csv('data/url2title.csv', delimiter='§', engine='python', usecols=['url'])

dfUrlTrue =[]
for x in list(dfUrlTrue_['url']):
    try:
        ext = tldextract.extract(x)
    except Exception as e:
        #print("Deu excessao no x:", x , "\n")
        continue
    if(ext.domain +'.'+ ext.suffix in list(trustyNewsDomains['url'])):
        dfUrlTrue.append(x)
    elif(ext.subdomain + "." + ext.domain +'.'+ ext.suffix in list(trustyNewsDomains['url'])):
        dfUrlTrue.append(x)
dfUrlTrue = pd.DataFrame(dfUrlTrue, columns = ['url'])




In [9]:
dfUrlTrue.head(1000)

,url
0,https://josiasdesouza.blogosfera.uol.com.br/
1,https://agorarn.com.br/
2,https://correio24horas.com.br/
3,https://tribunaonline.com.br/
4,https://correiodopovo.com.br/
...,...
995,https://www.nytimes.com/2021/08/04/books/cecil...
996,https://www.progresso.com.br/brasil/barbie-bra...
997,https://www.folhavitoria.com.br/cadernos/gerac...
998,https://www.gazetadopovo.com.br/vozes/lucio-vaz/


In [10]:
dfUrlTrue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257991 entries, 0 to 257990
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     257991 non-null  object
dtypes: object(1)
memory usage: 2.0+ MB


In [11]:
from source_features.source_features import SourceFeaturesExtractor

In [12]:
sourceExtractor = SourceFeaturesExtractor("8c4b00a0873a12e53b7a2384ad8eeb9f")

In [37]:
#Eu to deixando essa seção aqui pra testar rapidamente modificações no calculo das features..
#Na hora da coleta tu sempre descobre algo que ta fazendo errado ou pode melhorar

# url = "https://www.correiodopovo.com.br/"
    
# data_ipstack = sourceExtractor.get_ipstack_data(url)
# print(data_ipstack)
# print("\n")

# data_asn = sourceExtractor.get_asn_data(data_ipstack['subdomain_ip'])
# print(data_asn)
# print("\n")

# data_meta = sourceExtractor.get_ip_vs_asn_data(data_asn, data_ipstack)
# print(data_meta)
# print("\n")

# data_network = sourceExtractor.get_network_data(url)
# print(data_network)
# print("\n")

{'subdomain_ip': '179.191.187.66', 'subdomain_ip_cc': 'BR', 'subdomain_ip_is_brazil': True, 'subdomain_ip_is_us': False, 'subdomain_ip_latitude': -30.04, 'subdomain_ip_longitude': -51.23}


{'subdomain_as_n': '52580', 'subdomain_as_cc': 'BR'}


{'subdomain_ipcc_equal_ascc': True}


{'domain_route_hops': 14, 'domain_dns_caa_txt_count': 7}




In [16]:
sourceColumns = [
                   'subdomain_ip', 'subdomain_ip_cc','subdomain_ip_is_brazil','subdomain_ip_is_us',
                   'subdomain_ip_latitude','subdomain_ip_longitude',
                   'subdomain_as_n','subdomain_as_cc',
                   'subdomain_ipcc_equal_ascc',
                   'domain_route_hops','domain_dns_caa_txt_count'
                 ]

In [17]:
dfSourceFeatures = pd.DataFrame(columns = ['desinformacao_label', 'url'] + sourceColumns)

In [18]:
#Criando esse df a mais pq muitas urls compartilham as mesmas source features ja que sao do mesmo site
#é imperativo: as features em data_network demoram 1 minuto POR INSTANCIA para serem coletadas
    #300mil minutos recalculando vs 400 minutos reutilizando
    
dfSubdomainSourceFeatures = pd.DataFrame(columns = ['desinformacao_label','subdomain'] + sourceColumns)

In [19]:
def generate_source_features(dfUrlX, desinformacao_label):
    for index, row in dfUrlX.iterrows():
        #Esse if ta aqui só pra não calcular pra todos os dados o tempo todo enquanto desenvolvia o nb
        #Só comentar esse if pra percorrer todas as URLs calculando features
        if(index < 50):
            url = row['url']
            dissectedUrl = tldextract.extract(url)
            subdomain = '.'.join(part for part in dissectedUrl if part)

            #print("Url:", url)
            #print("subdomain:", subd)

            dfSourceFeatures.at[len(dfSourceFeatures),'desinformacao_label'] = desinformacao_label
            dfSourceFeatures.at[len(dfSourceFeatures)-1,'url'] = url

            if(subdomain in list(dfSubdomainSourceFeatures['subdomain'])):
                features = dfSubdomainSourceFeatures[dfSubdomainSourceFeatures['subdomain'] == subdomain][0:1]
                for col in sourceColumns:
                    dfSourceFeatures.at[len(dfSourceFeatures)-1,col] = features[0:1][col].values[0]
            else:
                #IPStack data vai ser calculado em cima do subdominio
                row_data_ipstack = sourceExtractor.get_ipstack_data(url)
                #ASN data vai ser calculado em cima do IP encontrado do SUBDOMINIO (IP de g1.globo.com e não globo.com)
                row_data_asn = sourceExtractor.get_asn_data(row_data_ipstack['subdomain_ip'])
                row_data_meta = sourceExtractor.get_ip_vs_asn_data(row_data_asn, row_data_ipstack)
                #Network data vai ser calculado em cima do DOMINIO (globo.com e não g1.globo.com)
                row_data_network = sourceExtractor.get_network_data(url)


                mergedDict = {**row_data_ipstack, **row_data_asn, **row_data_network,**row_data_meta}
                dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures),'desinformacao_label'] = desinformacao_label
                dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures-1),'subdomain'] = subdomain

                for col in sourceColumns:
                    dfSourceFeatures.at[len(dfSourceFeatures)-1,col] = mergedDict[col]
                    dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures)-1,col] = mergedDict[col]

In [20]:
generate_source_features(dfUrlTrue,0)

In [21]:
dfSourceFeatures.head(1000)

,desinformacao_label,url,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count
0,0,https://josiasdesouza.blogosfera.uol.com.br/,2600:9000:208f:e600:15:17d9:d540:93a1,US,False,True,47.6,-122.33,16509,US,True,12,59
1,0,https://agorarn.com.br/,170.81.43.64,BR,True,False,-26.96,-52.54,266400,BR,True,15,2
2,0,https://correio24horas.com.br/,204.199.44.209,BR,True,False,-19.92,-43.95,3549,US,False,12,3
3,0,https://tribunaonline.com.br/,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2
4,0,https://correiodopovo.com.br/,189.16.116.12,BR,True,False,-30.04,-51.23,4230,BR,True,14,7
5,0,https://agorarn.com.br/editoria/impresso,170.81.43.64,BR,True,False,-26.96,-52.54,266400,BR,True,15,2
6,0,https://www.correio24horas.com.br/index.php,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3
7,0,https://www.correio24horas.com.br/noticia/nid/...,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3
8,0,https://www.correio24horas.com.br/noticia/nid/...,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3
9,0,https://www.correio24horas.com.br/,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3


In [22]:
dfSubdomainSourceFeatures.head(1000)

,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count
0,josiasdesouza.blogosfera.uol.com.br,2600:9000:208f:e600:15:17d9:d540:93a1,US,False,True,47.6,-122.33,16509,US,True,12,59
1,agorarn.com.br,170.81.43.64,BR,True,False,-26.96,-52.54,266400,BR,True,15,2
2,correio24horas.com.br,204.199.44.209,BR,True,False,-19.92,-43.95,3549,US,False,12,3
3,tribunaonline.com.br,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2
4,correiodopovo.com.br,189.16.116.12,BR,True,False,-30.04,-51.23,4230,BR,True,14,7
5,www.correio24horas.com.br,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3
6,www.correiodopovo.com.br,179.191.187.66,BR,True,False,-30.04,-51.23,52580,BR,True,14,7


In [23]:
generate_source_features(dfUrlFalse,1)

In [24]:
dfSourceFeatures.head(1000)

,desinformacao_label,url,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count
0,0,https://josiasdesouza.blogosfera.uol.com.br/,2600:9000:208f:e600:15:17d9:d540:93a1,US,False,True,47.6,-122.33,16509,US,True,12,59
1,0,https://agorarn.com.br/,170.81.43.64,BR,True,False,-26.96,-52.54,266400,BR,True,15,2
2,0,https://correio24horas.com.br/,204.199.44.209,BR,True,False,-19.92,-43.95,3549,US,False,12,3
3,0,https://tribunaonline.com.br/,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2
4,0,https://correiodopovo.com.br/,189.16.116.12,BR,True,False,-30.04,-51.23,4230,BR,True,14,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,http://www.naturalnews.com/046239_sunlight_alt...,2606:4700::6810:8746,US,False,True,37.78,-122.42,13335,US,True,9,2
96,1,http://www.naturalnews.com/046772_hemp_seeds_t...,2606:4700::6810:8746,US,False,True,37.78,-122.42,13335,US,True,9,2
97,1,https://www.naturalnews.com/2019-07-12-climate...,2606:4700::6810:8746,US,False,True,37.78,-122.42,13335,US,True,9,2
98,1,https://www.naturalnews.com/2019-04-10-cancer-...,2606:4700::6810:8746,US,False,True,37.78,-122.42,13335,US,True,9,2


In [25]:
dfSubdomainSourceFeatures.head(1000)

,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count
0,josiasdesouza.blogosfera.uol.com.br,2600:9000:208f:e600:15:17d9:d540:93a1,US,False,True,47.6,-122.33,16509,US,True,12,59
1,agorarn.com.br,170.81.43.64,BR,True,False,-26.96,-52.54,266400,BR,True,15,2
2,correio24horas.com.br,204.199.44.209,BR,True,False,-19.92,-43.95,3549,US,False,12,3
3,tribunaonline.com.br,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2
4,correiodopovo.com.br,189.16.116.12,BR,True,False,-30.04,-51.23,4230,BR,True,14,7
5,www.correio24horas.com.br,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,11,3
6,www.correiodopovo.com.br,179.191.187.66,BR,True,False,-30.04,-51.23,52580,BR,True,14,7
7,www.jornaldacidadeonline.com.br,179.191.187.66,BR,True,False,-30.04,-51.23,52580,BR,True,9,8
8,revistaoeste.com,2606:4700:20::681a:2c7,US,False,True,37.78,-122.42,13335,US,True,9,3
9,terrabrasilnoticias.com,138.197.229.4,US,False,True,40.76,-73.98,14061,US,True,11,2


In [26]:
dfSourceFeatures['desinformacao_label']= dfSourceFeatures['desinformacao_label'].astype('bool')
#dfSourceFeatures['url']= dfSourceFeatures['url'].astype('')

#dfSourceFeatures['subdomain_ip']= dfSourceFeatures['subdomain_ip'].astype('')
dfSourceFeatures['subdomain_ip_cc']= dfSourceFeatures['subdomain_ip_cc'].astype('category')
dfSourceFeatures['subdomain_ip_is_brazil']= dfSourceFeatures['subdomain_ip_is_brazil'].astype('bool')
dfSourceFeatures['subdomain_ip_is_us']= dfSourceFeatures['subdomain_ip_is_us'].astype('bool')
dfSourceFeatures['subdomain_ip_latitude']= dfSourceFeatures['subdomain_ip_latitude'].astype('float')
dfSourceFeatures['subdomain_ip_longitude']= dfSourceFeatures['subdomain_ip_longitude'].astype('float')
#dfSourceFeatures['subdomain_as_n']= dfSourceFeatures['subdomain_as_n'].astype('')
dfSourceFeatures['subdomain_as_cc']= dfSourceFeatures['subdomain_as_cc'].astype('category')
dfSourceFeatures['subdomain_ipcc_equal_ascc']= dfSourceFeatures['subdomain_ipcc_equal_ascc'].astype('bool')
dfSourceFeatures['domain_route_hops']= dfSourceFeatures['domain_route_hops'].astype('int')
dfSourceFeatures['domain_dns_caa_txt_count']= dfSourceFeatures['domain_dns_caa_txt_count'].astype('int')

In [27]:
#dfSubdomainSourceFeatures['subdomain']= dfSubdomainSourceFeatures['subdomain'].astype('')

#dfSubdomainSourceFeatures['subdomain_ip']= dfSubdomainSourceFeatures['subdomain_ip'].astype('')
dfSubdomainSourceFeatures['subdomain_ip_cc']= dfSubdomainSourceFeatures['subdomain_ip_cc'].astype('category')
dfSubdomainSourceFeatures['subdomain_ip_is_brazil']= dfSubdomainSourceFeatures['subdomain_ip_is_brazil'].astype('bool')
dfSubdomainSourceFeatures['subdomain_ip_is_us']= dfSubdomainSourceFeatures['subdomain_ip_is_us'].astype('bool')
dfSubdomainSourceFeatures['subdomain_ip_latitude']= dfSubdomainSourceFeatures['subdomain_ip_latitude'].astype('float')
dfSubdomainSourceFeatures['subdomain_ip_longitude']= dfSubdomainSourceFeatures['subdomain_ip_longitude'].astype('float')
#dfSubdomainSourceFeatures['subdomain_as_n']= dfSubdomainSourceFeatures['subdomain_as_n'].astype('')
dfSubdomainSourceFeatures['subdomain_as_cc']= dfSubdomainSourceFeatures['subdomain_as_cc'].astype('category')
dfSubdomainSourceFeatures['subdomain_ipcc_equal_ascc']= dfSubdomainSourceFeatures['subdomain_ipcc_equal_ascc'].astype('bool')
dfSubdomainSourceFeatures['domain_route_hops']= dfSubdomainSourceFeatures['domain_route_hops'].astype('int')
dfSubdomainSourceFeatures['domain_dns_caa_txt_count']= dfSubdomainSourceFeatures['domain_dns_caa_txt_count'].astype('int')



In [28]:
dfSourceFeatures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   desinformacao_label        100 non-null    bool    
 1   url                        100 non-null    object  
 2   subdomain_ip               100 non-null    object  
 3   subdomain_ip_cc            100 non-null    category
 4   subdomain_ip_is_brazil     100 non-null    bool    
 5   subdomain_ip_is_us         100 non-null    bool    
 6   subdomain_ip_latitude      100 non-null    float64 
 7   subdomain_ip_longitude     100 non-null    float64 
 8   subdomain_as_n             100 non-null    object  
 9   subdomain_as_cc            100 non-null    category
 10  subdomain_ipcc_equal_ascc  100 non-null    bool    
 11  domain_route_hops          100 non-null    int64   
 12  domain_dns_caa_txt_count   100 non-null    int64   
dtypes: bool(4), category(2), float64(2),

In [29]:
#shuffle no df
dfSourceFeatures = dfSourceFeatures.sample(frac=1).reset_index(drop=True)
import pickle
dfSourceFeatures.to_pickle("./dfSourceFeatures.pkl")

In [30]:
# allSubs = []
# for index, row in dfUrlTrue.iterrows():
#     url = row['url']
#     dissectedUrl = tldextract.extract(url)
#     subdomain = '.'.join(part for part in dissectedUrl if len(part)>0)
#     allSubs.append(subdomain)
#     #print(len(allSubs))
# print(len(list(set(allSubs))))

# allSubs = []
# for index, row in dfUrlFalse.iterrows():
#     url = row['url']
#     dissectedUrl = tldextract.extract(url)
#     subdomain = '.'.join(part for part in dissectedUrl if len(part)>0)
#     allSubs.append(subdomain)
#     #print(len(allSubs))
# print(len(list(set(allSubs))))

In [31]:
# def lendo_dataset(localizacao_arquivo):
#     # Leitura de dados:
#     dados = []
#     with open(localizacao_arquivo, mode='r') as f:
#         for line in f:
#             dados.append(json.loads(line.strip()))

#     # Imprimindo primeiro 3 coletas de cada
#     for i in range(3):
#         pp = pprint.PrettyPrinter(indent=4)
#         pp.pprint(dados[i])
#         print('\n')


# # Lendo TrueNews
# print('\n---------------TRUE NEWS:\n')
# lendo_dataset('data/DATASET_MPMG-TrueNews_selected.txt')

# print('\n\n\n---------------FAKE NEWS:\n')
# # Lendo FakeNews
# lendo_dataset('data/DATASET_MPMG-FakeNews_matched.txt')
# dfTrue = pd.read_json('data/DATASET_MPMG-TrueNews_selected.json')
# dfFalse = pd.read_json('data/DATASET_MPMG-FakeNews_matched.json')
# print(dfTrue.columns)

# dfTrue.replace('', np.nan, inplace=True)
# dfFalse.replace('', np.nan, inplace=True)
# dfTrue.replace('NULL', np.nan, inplace=True)
# dfFalse.replace('NULL', np.nan, inplace=True)
#print(dfTrue.shape, dfFalse.shape)